In [1]:
# ПЕРВЫЙ ВАРИАНТ ДЛЯ ЗАДАЧИ 1
# with open("city_weather.csv", "r", encoding="UTF-8") as f:
#     city_list = []
#     next(f)
#     for i, line in enumerate(f):
#         sity, date, day, nigth = line.strip().split(",")
#         print(f"{i}. {sity} - {date}- {day} - {nigth} - {(float(day) + float(nigth)) / 2}")
#         city_list.append((sity, (float(day) + float(nigth)) / 2, 1))
#
# city_list = sorted(city_list, key=lambda x: x[0])
# print(city_list)
#
# cur_city = None
# count_temp = 0
# count = 0
# for city in city_list:
#     one = city[1]
#
#     if cur_city:
#         if cur_city == city[0]:
#             count_temp += one
#             count += city[2]
#         else:
#             print(cur_city, count_temp, count_temp / count)
#             count = 0
#             count_temp = one
#             cur_city = city[0]
#     else:
#         cur_city = city[0]
#         count_temp = one
#
# print(cur_city, count_temp)

In [ ]:
import csv
from collections import defaultdict


def mapper_temperature(input_file):
    for row in input_file:
        city = row['city']
        night_temp = float(row['night_temperature'])
        day_temp = float(row['day_temperatre'])
        avg_temp = (night_temp + day_temp) / 2
        yield city, (avg_temp, 1)  # (сумма температур, количество)


def mapper_region(input_file):
    city_region_map = {}
    for row in input_file:
        region = row['federal_district']
        city = row['city']
        city_region_map[city] = region
    return city_region_map


def reducer(mapped_data):
    city_temps = defaultdict(lambda: [0.0, 0])  # (сумма температур, количество)

    for city, (temp_sum, count) in mapped_data:
        city_temps[city][0] += temp_sum  # Суммируем температуры по городам
        city_temps[city][1] += count  # Считаем количество записей по городам

    return city_temps


def calculate_average_and_difference(city_temps, city_region_map):
    region_totals = defaultdict(lambda: [0.0, 0])  # (сумма температур по округу, количество)

    for city, (total_temp, count) in city_temps.items():
        if city in city_region_map:
            region = city_region_map[city]
            region_totals[region][0] += total_temp  # Суммируем температуры по округам
            region_totals[region][1] += count  # Считаем количество записей по округам

            average_city_temp = total_temp / count if count > 0 else 0
            print(f"{city} (Город) - Средняя температура: {average_city_temp:.2f}")

    for region, (total_temp, count) in region_totals.items():
        average_region_temp = total_temp / count if count > 0 else 0
        print(f"{region} (Округ) - Средняя температура: {average_region_temp:.2f}")

        # Вычисляем разницу между средней температурой города и округа
        for city in city_temps.keys():
            if city in city_region_map and city_region_map[city] == region:
                average_city_temp = city_temps[city][0] / city_temps[city][1]
                difference = average_city_temp - average_region_temp
                print(f"Разница для {city} в {region}: {difference:.2f}")


def main():
    with open('city_weather.csv', mode='r', encoding='utf-8') as temp_file:
        temp_reader = csv.DictReader(temp_file)
        temperature_data = list(mapper_temperature(temp_reader))  # Применяем маппер для температур

    with open('district_to_city.csv', mode='r', encoding='utf-8') as region_file:
        region_reader = csv.DictReader(region_file)
        city_region_map = mapper_region(region_reader)  # Получаем соответствие городов и округов

    # Суммируем температуры и считаем количество по городам
    city_temps = reducer(temperature_data)

    # Вычисляем средние температуры и разницу между ними
    calculate_average_and_difference(city_temps, city_region_map)


if __name__ == "__main__":
    main()